### https://zhuanlan.zhihu.com/p/176241946

In [1]:
import datetime
import backtrader as bt
import backtrader.indicators as bi
# import backtest
import pandas as pd
import numpy as np
# import math
# import matplotlib.pyplot as plt

In [2]:
from lutils.stock import LTdxHq

In [3]:
class GridStrategy(bt.Strategy):
    params = (
        ("printlog", True),
        ("top", 12),
        ("buttom", 7),
    )
    
    def __init__(self):
        self.mid = (self.p.top + self.p.buttom)/2.0
        # 百分比区间计算
        #这里多1/2，是因为arange函数是左闭右开区间。
        perc_level = [x for x in np.arange(1 + 0.02 * 5, 1 - 0.02 * 5 - 0.02/2, -0.02)]
        # 价格区间
        # print(self.mid)
        self.price_levels = [self.mid * x for x in perc_level]
        # 记录上一次穿越的网格
        self.last_price_index = None
        # 总手续费
        self.comm = 0.0
 
    def next(self):
        # print(self.last_price_index)
        # 开仓
        if self.last_price_index == None:
            # print("b", len(self.price_levels))
            for i in range(len(self.price_levels)):
                price = self.data.close[0]
                # print("c", i, price, self.price_levels[i][0])
                if self.data.close[0] > self.price_levels[i]:
                    self.last_price_index = i
                    self.order_target_percent(target=i/(len(self.price_levels) - 1))
                    print("a")
                    return
        # 调仓
        else:
            signal = False
            while True:
                upper = None
                lower = None
                if self.last_price_index > 0:
                    upper = self.price_levels[self.last_price_index - 1]
                if self.last_price_index < len(self.price_levels) - 1:
                    lower = self.price_levels[self.last_price_index + 1]
                # 还不是最轻仓，继续涨，再卖一档
                if upper != None and self.data.close > upper:
                    self.last_price_index = self.last_price_index - 1
                    signal = True
                    continue
                # 还不是最重仓，继续跌，再买一档
                if lower != None and self.data.close < lower:
                    self.last_price_index = self.last_price_index + 1
                    signal = True
                    continue
                break
            if signal:
                self.long_short = None
                self.order_target_percent(target=self.last_price_index/(len(self.price_levels) - 1))
 
    # 输出交易记录
    def log(self, txt, dt = None, doprint = False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))
 
    def notify_order(self, order):
        # 有交易提交/被接受，啥也不做
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 交易完成，报告结果
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('执行买入, 价格: %.2f, 成本: %.2f, 手续费 %.2f' % (order.executed.price, order.executed.value, order.executed.comm))
                self.buyprice = order.executed.price
                self.comm += order.executed.comm
            else:
                self.log('执行卖出, 价格: %.2f, 成本: %.2f, 手续费 %.2f' % (order.executed.price, order.executed.value, order.executed.comm))
                self.comm += order.executed.comm
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("交易失败")
        self.order = None
 
    # 输出手续费
    def stop(self):
        self.log("手续费:%.2f 成本比例:%.5f" % (self.comm, self.comm/self.broker.getvalue()))



In [4]:
lt = LTdxHq()

In [5]:
code = '600217' # 510300 000032 300142 603636 600519 688567 600217

In [6]:
df = lt.get_k_data_daily(code, qfq=True)

In [7]:
# df = df['2021-01-01':]

In [8]:
top = df.close.min() * .8
buttom = df.close.max() * 1.1

In [9]:
top

0.784

In [10]:
buttom

16.896

In [11]:
df.index = pd.to_datetime(df.index)

In [12]:
cerebro = bt.Cerebro(oldtrades=True)

feed = bt.feeds.PandasData(dataname=df, openinterest=None, compression=1, timeframe=bt.TimeFrame.Minutes)
cerebro.adddata(feed)

# top = 4.2
# buttom = 3.5
cerebro.addstrategy(GridStrategy, top=top, buttom=buttom)

# 小场面1万起始资金
cerebro.broker.setcash(10000.0)

# 手续费万5
cerebro.broker.setcommission(0.0005)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

result = cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
a
2001-06-25, 执行买入, 价格: 8.28, 成本: 9008.64, 手续费 4.50
2001-06-26, 执行卖出, 价格: 8.48, 成本: 1026.72, 手续费 0.53
2001-06-27, 执行卖出, 价格: 8.77, 成本: 2036.88, 手续费 1.08
2001-06-28, 执行卖出, 价格: 9.07, 成本: 2020.32, 手续费 1.11
2001-06-29, 执行卖出, 价格: 9.39, 成本: 2003.76, 手续费 1.14
2001-07-02, 执行卖出, 价格: 9.68, 成本: 977.04, 手续费 0.57
2001-07-03, 执行卖出, 价格: 9.88, 成本: 943.92, 手续费 0.56
2001-07-04, 执行买入, 价格: 8.26, 成本: 4072.18, 手续费 2.04
2001-07-05, 执行买入, 价格: 8.09, 成本: 3462.52, 手续费 1.73
2001-07-06, 执行买入, 价格: 7.51, 成本: 3394.52, 手续费 1.70
2007-05-24, 执行卖出, 价格: 8.64, 成本: 1082.57, 手续费 0.59
2007-05-28, 执行买入, 价格: 7.81, 成本: 1171.50, 手续费 0.59
2007-10-08, 执行卖出, 价格: 8.25, 成本: 1088.31, 手续费 0.57
2007-10-09, 执行买入, 价格: 7.65, 成本: 1124.55, 手续费 0.56
2011-04-11, 执行卖出, 价格: 8.50, 成本: 1084.07, 手续费 0.58
2011-04-12, 执行卖出, 价格: 8.25, 成本: 1091.99, 手续费 0.57
2011-04-13, 执行卖出, 价格: 8.74, 成本: 2286.84, 手续费 1.26
2011-04-14, 执行卖出, 价格: 9.05, 成本: 2255.19, 手续费 1.29
2011-04-15, 执行卖出, 价格: 9.34, 成本: 1147.38, 手续费 0.68
2011-04-19, 执行买

In [13]:
cerebro.plot(
    iplot=False,
    start=datetime.date(2020, 1, 1),
    end=datetime.date(2021, 9, 30),
    style='candlestick',
    barup='red',
    bardown='green',
)

[[<Figure size 1267x829 with 4 Axes>]]

In [8]:
# start = "2018-01-01"
# end = "2021-07-05"
# name = ["300etf"]
# code = ["510300"]
# backtest = backtest.BackTest(GridStrategy, start, end, code, name, 100000)
# result = backtest.run()
# # backtest.output()
# print(result)